Here we'll learn to prect type of recipe by it's instruction! 

This approach can be used for automatical recipies tagging by categories.

![img](https://www.restoconnection.com/wp-content/uploads/2017/05/instagram-food.jpg)

imports for all cases of life:

In [ ]:
import sys, json, codecs, csv
import numpy as np
import random
import pandas as pd
import tqdm
import itertools
import seaborn as sns

# visualization
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

nltk imports for text preprocessing

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

imports for model training

In [ ]:
from keras.layers import Dense, Activation
from keras.layers import LSTM, GRU, Dropout, BatchNormalization
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import RMSprop, Adam

# word2vec
from gensim.models import word2vec

read database with recipies

In [ ]:
with codecs.open('./edimdoma_dataset.csv', 'r') as f:
    reader = csv.reader(f)

    names = []
    descriptions = []
    categories = []
    instructions = []
    cookTimes = []
    for line_id, line in enumerate(reader):
        if line_id == 0:
            continue
        doc = line[2]
        recipe_info = json.loads(doc)

        cookTime = -1
        if 'cookTime' in recipe_info:
            cookTime = recipe_info['cookTime']
            if cookTime.startswith('PT'):
                cookTime = cookTime[2:]
            if cookTime.endswith('H'):
                cookTime = cookTime[:-1]
            if cookTime.isdigit():
                cookTime = int(cookTime)
                
        if not 'recipeCategory' in recipe_info:
            continue
        category = recipe_info['recipeCategory'].strip() if 'recipeCategory' in recipe_info else 'NONE'
        cuisine = recipe_info['recipeCuisine'].strip() if 'recipeCuisine' in recipe_info else 'NONE'
        name = recipe_info['name'].strip()
        description = recipe_info['description'].strip()
        recipeIngredient = recipe_info['recipeIngredient']
        recipeInstructions = [x.strip() for x in recipe_info['recipeInstructions'] if not x == None]

        names.append(name)
        descriptions.append(description)
        categories.append(category)
        instructions.append(' '.join(recipeInstructions))
        cookTimes.append(cookTime)

print categories values

In [ ]:
plt.figure(figsize=(12,8))
sns.set(style="darkgrid")
ax = sns.countplot(categories)

ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.tight_layout()
plt.show()

prepare map category => id for model training

In [ ]:
cats = sorted(set(categories))
cat2id = {cat:index for index, cat in enumerate(cats)}

In [ ]:
stop_words = stopwords.words('russian')
stop_words.extend(['.', ';', '!' ])
print(u" ".join(stop_words))

Let's prepare texts for training

In [ ]:
sentences = <prepare sentences for word2vec training. You should: 
    1. concatenate some instructions in one big string;
    2. Tokenize text by sentences;
    3. Tokenize each sentence by words;
    4. remove stop words and perform lovercasing>

# Train word2vec

Ok, word2vec will be needed for us to train a deep model. Let's prepare it!

In [ ]:
trained_word2vec = <train gensim word2vec with your sentences> 

Let's review our embeddings!

In [ ]:
for x in trained_word2vec.similar_by_word(u'мясо'):
    print(u"{}: {}".format(x[0], x[1]))

In [ ]:
for x in trained_word2vec.similar_by_word(u'порубить'):
    print(u"{}: {}".format(x[0], x[1]))

Let's try to do that stuff manually!

In [ ]:
from numpy.linalg import norm
<write your own similar_by_word function>

Good stuff!

# Training the category classifier

Ok, let's try to prepare a model which will classify our texts:

In [ ]:
print('Build model...')
input = Input(shape=(None, 300))
out = GRU(512, return_sequences=True)(input)
out = <Your code here>verything is great and working with Victor is a pleasure.
out = LSTM(128)(out)
out = Dense(len(cat2id), activation='softmax')(out)
out = <Your code here>
model = Model(input, out)
model.compile("adam", loss='categorical_crossentropy')

Don't forget to split your dataset on train/test if you don't want to overfit!

In [ ]:
dataset = list(zip(*[instructions, categories]))
random.shuffle(dataset)
train_dataset = dataset[:int(len(dataset)*.7)]
test_dataset = dataset[int(len(dataset)*.7):]

batches generator

In [ ]:
def get_batches(dataset, batch_count=64, batch_size=64, seq_size=20):
    batch_x = np.zeros((batch_size, seq_size, 300))
    batch_y = np.zeros((batch_size, len(cat2id)))
    for bi in range(batch_count):
        for seq_index in range(batch_size):
            text, target = random.choice(dataset)
            for i in range(seq_size):
                batch_x[seq_index,i] = <initialize batch_x with prepared word2vec values>
            batch_y[seq_index, :] = <initialize label>
        yield batch_x, batch_y

Let's star iterative training process:

In [ ]:
for epoch in range(100):
    print("epoch: {}".format(epoch))
    losses = []
    for batch_x, batch_y in get_batches(train_dataset):
        loss = model.train_on_batch(batch_x, batch_y)
        losses.append(loss)
    print("train_loss: {}".format(np.mean(losses)))
    
    losses = []
    for batch_x, batch_y in get_batches(test_dataset):
        loss = model.test_on_batch(batch_x, batch_y)
        losses.append(loss)
    print("test_loss: {}".format(np.mean(losses)))

# Check the model

We've trained the our model, but we don't know how it works in production.
Let's check it!

In [ ]:
def predict(query):
    seq_size = 40
    batch = np.zeros((seq_size, 300))
    words = [x for x in word_tokenize(query) if not x in stop_words and x.isalnum()]
    for i in range(seq_size):
        batch[i,:] = 0
        if i < len(words) and words[i] in trained_word2vec:
            batch[i,:] = trained_word2vec[words[i]]
    preds = model.predict(np.expand_dims(batch, 0))[0]
    for x in sorted(enumerate(preds), key=lambda x: x[1], reverse=True)[:5]:
        print(u"{}: {}".format(cats[x[0]], x[1]))

In [ ]:
query, category = test_dataset[-12]
print("----------------------------")
print(query)
print("----------------------------")
print(category)
print("----------------------------")
print("prediction:")
predict(query)

In [ ]:
query, category = test_dataset[-10]
print("----------------------------")
print(query)
print("----------------------------")
print(category)
print("----------------------------")
print("prediction:")
predict(query)

We've done that!

# Homework:

1. **5 points**: try to make the model significantly better. You can add any crazy stuff and tune any hyperparameters which come to mind (e.g sequence size, change number of gru/lstm layers, word2vec iterations etc etc). There is a huge number of parameters!
2. **2 points**: learn more about batchnorm (https://arxiv.org/abs/1502.03167) and dropout (https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) layers. Try to apply them here. What is better?
3. **3 points**: solve another problem - prediction of cooking time ('cookTime' field). It's regression problem. You should to think about activation function of last layer and loss function